
# 🧠 House Prices 回归建模学习笔记（详细版 by ChatGPT）

---

## 📦 项目背景

这是 Kaggle 上经典的房价预测竞赛。数据集中有 79 个住宅属性（特征），我们需要通过特征工程 + 建模技术预测最终房价。目标是练习：

- 数据清洗与分析（EDA）
- 特征工程（定量/定性变量转换、编码）
- 模型构建（Lasso、XGBoost、LGBM）

---

## 🧮 1. Pandas 数据类型与定量/定性划分

```python
quantitative = [f for f in train.columns if train.dtypes[f] != 'object']
qualitative = [f for f in train.columns if train.dtypes[f] == 'object']
```

- `object` 在 pandas 中表示字符串类型，通常是分类变量（定性）
- `int64`, `float64` 是数值型，表示定量变量

---

## 📉 2. 缺失值分析与可视化

```python
missing = train.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar()
```

**详细解释：**
- `isnull().sum()` 得到每列的缺失值数量
- `missing[missing > 0]` 过滤掉没有缺失值的列
- `sort_values()` 让柱状图更清晰
- 最终画出哪些变量缺失最多（如 Fence, PoolQC）

---

## 🧪 3. 正态性检验

```python
test_normality = lambda x: stats.shapiro(x.fillna(0))[1] < 0.01
normal = train[quantitative].apply(test_normality)
print(not normal.any())  # False
```

**解释：**
- 使用 Shapiro-Wilk 检验判断一个特征是否服从正态分布
- 若 p 值 < 0.01，说明拒绝“正态分布”的假设
- `not normal.any()` 表示是否“所有变量都通过正态检验”，结果 False 表示“至少一个变量不服从正态分布”
- 实际上，几乎所有变量都不满足

---

## 📈 4. 分布拟合：JohnsonSU / Normal / LogNormal

```python
sns.distplot(y, kde=False, fit=stats.johnsonsu)
sns.distplot(y, kde=False, fit=stats.norm)
sns.distplot(y, kde=False, fit=stats.lognorm)
```

**解释：**
- 用不同的概率分布函数来拟合房价 SalePrice 的分布
- `fit=...` 表示让 scipy 用最大似然估计（MLE）找出最优参数
- Johnson SU 分布最灵活，适合非对称数据

---

## 📌 5. 最大似然估计（MLE）

> 寻找使得观测数据在该分布下概率最大的参数估计方法

数学形式：

\[
\hat{\theta} = \arg\max_\theta \prod_{i=1}^n f(x_i | \theta)
\]

- 通常取对数简化计算（对数似然）
- 拟合过程由 scipy 完成，无需手工计算

---

## 📊 6. Spearman 相关性

```python
frame[f].corr(frame['SalePrice'], method='spearman')
```

- 对变量进行“排名”后再计算 Pearson 相关系数
- 不要求变量是线性关系或正态分布
- 非常适合用于房价预测中离散变量（如卧室数量）

---

## 🔄 7. 定性变量编码（目标编码）

```python
def encode(frame, feature):
    ordering = pd.DataFrame()
    ordering['val'] = frame[feature].unique()
    ordering.index = ordering.val
    ordering['spmean'] = frame[[feature, 'SalePrice']].groupby(feature).mean()['SalePrice']
    ordering = ordering.sort_values('spmean')
    ordering['ordering'] = range(1, ordering.shape[0]+1)
    ordering = ordering['ordering'].to_dict()
    for cat, o in ordering.items():
        frame.loc[frame[feature] == cat, feature+'_E'] = o
```

**详细解释：**
1. 找出每个类别（如不同 Neighborhood）
2. 计算该类别的平均房价
3. 按照平均房价从低到高编号：例如 `{'Crawfor': 4, 'CollgCr': 3, ...}`
4. 将原分类变量转化为有等级含义的数值变量，便于模型处理

---

## 📉 8. 相关性排序图

```python
def spearman(frame, features):
    spr = pd.DataFrame()
    spr['feature'] = features
    spr['spearman'] = [frame[f].corr(frame['SalePrice'], 'spearman') for f in features]
    spr = spr.sort_values('spearman')
    sns.barplot(data=spr, y='feature', x='spearman', orient='h')
```

- 分析哪些变量与房价最相关
- 相关性越高的变量越适合建模使用
- 可帮助特征选择、降维

---

## 🧠 认知提升：数学 vs 编程

| 编程作用      | 数学作用                          |
|---------------|-----------------------------------|
| 构建系统      | 抽象结构，洞察变量关系            |
| 执行建模过程  | 推理模型边界，理解分布与优化目标  |
| 数据变换实现  | 判断是否需要变换，变换方向与方法  |

人类价值在于：
- 构造结构（如新变量）
- 解释变量含义（如 MSSubClass 实为分类）
- 决定方法使用（用 Spearman 而非 Pearson）

---

